In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchvision.datasets as datasets
import torchvision.transforms as transforms

from PIL import Image
import numpy as np
import time
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pickle

filePath = '/content/drive/MyDrive/aispark/서윤_trainset.txt'

with open(filePath, 'rb') as lf:
    readList = pickle.load(lf)
    print(readList)
trainset = readList

In [ ]:
from random import shuffle

shuffle(trainset)
valset = trainset[4000:4500]
testset = trainset[4500:]
trainset = trainset[:4000]

partition = {'train': trainset, 'val':valset, 'test':testset}

In [ ]:
batch_size =32
learning_rate = 0.001
epochs = 10
image_size = 715
input_size = 100
label_size = 63

In [ ]:
train_loader = torch.utils.data.DataLoader(partition['train'], 
                                              batch_size=32, 
                                              shuffle=True, num_workers=2)

dataiter = iter(train_loader)

images, labels = dataiter.next()

images.size() #배치사이즈:32,  이미지 너비: 100, 시퀀스 크기 : 715

torch.Size([32, 100, 715])

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [ ]:
class BLSTMATT(nn.Module): # 100, 
    def __init__(self, input_size, label_size, batch_size, num_layer=2):
        super(BLSTMATT, self).__init__()
        self.blstm = torch.nn.LSTM(input_size, input_size, num_layer, bidirectional=True, batch_first=True)
        self.h0 = torch.zeros(2 * num_layer, batch_size, input_size).cuda()
        self.c0 = torch.zeros(2 * num_layer, batch_size, input_size).cuda()
        self.softmax = nn.Softmax(dim=2)
        self.tanh = nn.Tanh()
        self.batch_size = batch_size
        self.hidden_size = input_size 
        self.loss = nn.CrossEntropyLoss()
        self.w = torch.zeros(input_size).cuda()

        self.embedding_dropout = nn.Dropout(0.3)
        self.lstm_dropout = nn.Dropout(0.3)
        self.attention_dropout = nn.Dropout(0.5)

        self.fc = nn.Sequential(nn.Linear(input_size, label_size))

    def Att_layer(self, H):
        M = self.tanh(H)
        alpha = self.softmax(torch.bmm(M, self.w.repeat(self.batch_size, 1, 1).transpose(1, 2)))
        res = self.tanh(torch.bmm(alpha.transpose(1,2), H))
        return res

    def forward(self, x_input):
        x_input = self.embedding_dropout(x_input)
        h, _ = self.blstm(x_input, (self.h0, self.c0))
        h = h[:,:,self.hidden_size:] + h[:,:,:self.hidden_size]
        h = self.lstm_dropout(h)
        atth = self.Att_layer(h)
        atth = self.attention_dropout(atth)
        out = self.fc(atth)
        out = self.softmax(out)

        return out.view(self.batch_size, -1)
        print(out)

In [ ]:
model = BLSTMATT(100, 63, 32, 1).cuda()#input_size, label_size, batch_size, num_layer=2
model

BLSTMATT(
  (blstm): LSTM(100, 100, batch_first=True, bidirectional=True)
  (softmax): Softmax(dim=2)
  (tanh): Tanh()
  (loss): CrossEntropyLoss()
  (embedding_dropout): Dropout(p=0.3, inplace=False)
  (lstm_dropout): Dropout(p=0.3, inplace=False)
  (attention_dropout): Dropout(p=0.5, inplace=False)
  (fc): Sequential(
    (0): Linear(in_features=100, out_features=63, bias=True)
  )
)

In [ ]:
valid_loader =  torch.utils.data.DataLoader(partition['val'], 
                                              batch_size=32, 
                                              shuffle=False, num_workers=2,
                                            drop_last=True)


dataiter = iter(valid_loader)

images, labels = dataiter.next()

images.size() #배치사이즈:32,  이미지 너비: 100, 시퀀스 크기 : 715

torch.Size([32, 100, 715])

In [ ]:
train_loss_arr = []
valid_loss_arr = []

optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

for epoch in range(8):
    start = time.time()
    train_avg_loss = 0
    model.train()

    running_corrects = 0
    
    for image, label in train_loader:
        # ------- assign train data
        image = np.swapaxes(image,1,2 )

        x = image.to(device)
        x = torch.tensor(x, dtype=torch.float32)
        label = label.to(device)
        label = torch.tensor(label, dtype=torch.float32)
        # ------- forward prop
        optimizer.zero_grad()
        output = model(x.float())
        _, preds = torch.max(output, 1)
        # ------- backward prop
        loss = criterion(output.float().cuda(), label.type(torch.LongTensor).cuda())
        loss.backward()
        optimizer.step()
        # ------- get train performance
        
        train_avg_loss += loss / (len(train_loader))
        running_corrects += (preds.cuda() == label.data.cuda()).sum().item()
        running_accuracy = running_corrects / (len(train_loader))
    train_loss_arr.append(train_avg_loss)
    print(f'Epoch : {epoch+1}/{epochs}, train_loss : {train_avg_loss:.4f}, train_acc: {running_accuracy:.4f}', sep='\n')


    model.eval()
    with torch.no_grad():
      valid_avg_loss =0
      val_corrects = 0
      for image, label in valid_loader:
        # ------- assign valid data
        image = np.swapaxes(image,1,2 )
        image = image.to(device)
        label = label.to(device)
        # ------- forward prop
        val_output = model(image.float())
        _, val_preds = torch.max(val_output, 1)
        val_loss = criterion(val_output.float().cuda(), label.type(torch.LongTensor).cuda())
        # ------- get valid performance
        val_corrects += (val_preds.cuda() == label.data.cuda()).sum().item()
        val_accuracy = val_corrects / (len(valid_loader))
        valid_avg_loss += val_loss / (len(valid_loader)) # val_loss / total_Iteration
      valid_loss_arr.append(valid_avg_loss) 
      print(f'val_loss : {valid_avg_loss:.4f}, valid_acc : {val_accuracy:.4f}')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Epoch : 1/10, train_loss : 4.1424, train_acc: 0.4880
val_loss : 4.1428, valid_acc : 0.3333
Epoch : 2/10, train_loss : 4.1400, train_acc: 0.7680
val_loss : 4.1423, valid_acc : 0.4667
Epoch : 3/10, train_loss : 4.1399, train_acc: 0.7440
val_loss : 4.1414, valid_acc : 0.6667
Epoch : 4/10, train_loss : 4.1386, train_acc: 0.7600
val_loss : 4.1420, valid_acc : 0.8000
Epoch : 5/10, train_loss : 4.1396, train_acc: 0.7760
val_loss : 4.1434, valid_acc : 0.6667
Epoch : 6/10, train_loss : 4.1389, train_acc: 0.8480
val_loss : 4.1421, valid_acc : 0.6667
Epoch : 7/10, train_loss : 4.1389, train_acc: 0.7760
val_loss : 4.1416, valid_acc : 0.8000
Epoch : 8/10, train_loss : 4.1389, train_acc: 0.7200
val_loss : 4.1417, valid_acc : 0.8000


In [ ]:
test_loader =  torch.utils.data.DataLoader(partition['test'], 
                                              batch_size=32, 
                                              shuffle=False, num_workers=2,drop_last=True)


dataiter = iter(valid_loader)

images, labels = dataiter.next()

images.size() #배치사이즈:32,  이미지 너비: 100, 시퀀스 크기 : 715

torch.Size([32, 100, 715])

In [ ]:
with torch.no_grad():
    start = time.time()
    test_corrects=0
    test_avg_loss = 0
    model.eval()
    for image, label in test_loader:
        # ------- assign valid data
        image = np.swapaxes(image,1,2 )
        image = image.to(device)
        label = label.to(device)
        # ------- forward prop
        test_output = model(image.float())
        _, test_preds = torch.max(val_output, 1)
        test_loss = criterion(test_output.float().cuda(), label.type(torch.LongTensor).cuda())
        # ------- get valid performance
        test_corrects += (test_preds.cuda() == label.data.cuda()).sum().item()
        test_accuracy = test_corrects / (len(test_loader))
        test_avg_loss += test_loss / (len(test_loader))
    print(f'test_loss : {test_avg_loss:.4f}, test_acc : {test_accuracy:.4f}')

test_loss : 4.1401, test_acc : 0.6000
